<a href="https://colab.research.google.com/github/usm-cos422-522/courseMaterials/blob/main/Lectures/Chapter8-Lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chapter 8.3 : Step by step

### Long Format for data

A common way to store time series data is sometimes called long or stacked format. In this format, individual values are represented by a single row in a table rather than multiple values per row.

Wide Format
Region  Time-1   Time-2   Time-3
A         10       12       13
B          6        7       10

Long Format
Region Time       Value
A     Time-1     10
A     Time-2     12
A     Time-3     13
B     Time-1      6
B     Time-2      7
B     Time-3     10

But this Long format can apply to other non-time datasets as well

Wide Format:
Person   Age   Weight   Height  
 -------- ----- -------- -------- 
  Bob       32      168      180  
  Alice     24      150      175  
  Steve     64      144      165

Long Format:
Person   Variable   Value  
 -------- ---------- ------- 
  Bob      Age           32  
  Bob      Weight       168  
  Bob      Height       180  
  Alice    Age           24  
  Alice    Weight       150  
  Alice    Height       175  
  Steve    Age           64  
  Steve    Weight       144  
  Steve    Height       165

We focus on Time Series Data for now ... a lot if interest in prediction, commercial projection, $ ...

TLDR
    Wide --> Long
        stack with some work
        pd.melt
    Long --> Wide
        pivot

    pivot_table is a generalization of pivot, melt

In [1]:
import pandas as pd
import numpy as np

### Read and Prepare the Data

Select date columns and gdp, inflation, and unemployment data

In [2]:
data_url = "https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/macrodata.csv"
original_data = pd.read_csv(data_url)

In [3]:
original_data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [4]:
# We select a subset of the columns

wide_data = original_data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
wide_data.head()

,year,quarter,realgdp,infl,unemp
0,1959.0,1.0,2710.349,0.00,5.8
1,1959.0,2.0,2778.801,2.34,5.1
2,1959.0,3.0,2775.488,2.74,5.3
3,1959.0,4.0,2785.204,0.27,5.6
4,1960.0,1.0,2847.699,2.31,5.2


#### Create a date index by combining Year and Quarter

In [5]:
# PeriodIndex : Immutable ndarray holding ordinal values indicating regular periods in time.
pd.PeriodIndex?

In [6]:
periods = pd.PeriodIndex(year=wide_data.year, quarter=wide_data.quarter,name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [7]:
# Now we have the date for the beginning of the quarter.
wide_data.index = periods.to_timestamp("D")
wide_data.head()

,year,quarter,realgdp,infl,unemp
date,,,,,
1959-01-01,1959.0,1.0,2710.349,0.00,5.8
1959-04-01,1959.0,2.0,2778.801,2.34,5.1
1959-07-01,1959.0,3.0,2775.488,2.74,5.3
1959-10-01,1959.0,4.0,2785.204,0.27,5.6
1960-01-01,1960.0,1.0,2847.699,2.31,5.2


In [8]:
# We no long need the year and the quarter columns
wide_data = wide_data.loc[:, ["realgdp", "infl", "unemp"]]
wide_data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


### Move from the Wide format to the Long format
Extensible DB format. Can add new attributes without changing schema.
Format used in many extensible data collection tools
Sometimes called EAV (Entity-Attribute-Value) format


#### Use Panda's melt method to convert from Wide to Long format

In [15]:
long_data = pd.melt(wide_data.reset_index(),id_vars='date',var_name='attribute',value_name='value')
# the above reset_index is needed to bring date field to a column ... below will fail
# melted = pd.melt(wide_data,id_vars='date',var_name='attribute',value_name='value')
long_data

,date,attribute,value
0,1959-01-01,realgdp,2710.349
1,1959-04-01,realgdp,2778.801
2,1959-07-01,realgdp,2775.488
3,1959-10-01,realgdp,2785.204
4,1960-01-01,realgdp,2847.699
...,...,...,...
604,2008-07-01,unemp,6.000
605,2008-10-01,unemp,6.900
606,2009-01-01,unemp,8.100
607,2009-04-01,unemp,9.200


In [16]:
#above data frame has index ordered by attributes, suppose we want oreder by dates
long_data = long_data.sort_values('date')
long_data

,date,attribute,value
0,1959-01-01,realgdp,2710.349
203,1959-01-01,infl,0.000
406,1959-01-01,unemp,5.800
1,1959-04-01,realgdp,2778.801
204,1959-04-01,infl,2.340
...,...,...,...
404,2009-04-01,infl,3.370
201,2009-04-01,realgdp,12901.504
202,2009-07-01,realgdp,12990.341
405,2009-07-01,infl,3.560


In [17]:
long_data.index

Int64Index([  0, 203, 406,   1, 204, 407,   2, 205, 408,   3,
            ...
            199, 403, 200, 606, 607, 404, 201, 202, 405, 608],
           dtype='int64', length=609)

In [18]:
long_data = long_data.set_index('date')
long_data

,attribute,value
date,,
1959-01-01,realgdp,2710.349
1959-01-01,infl,0.000
1959-01-01,unemp,5.800
1959-04-01,realgdp,2778.801
1959-04-01,infl,2.340
...,...,...
2009-04-01,infl,3.370
2009-04-01,realgdp,12901.504
2009-07-01,realgdp,12990.341


#### Stacking can also be used to convert from Wide to Long
#### First try at making long data using stack


In [ ]:
long_data_messy = wide_data.stack()
long_data_messy[:10]

date        item   
1959-01-01  realgdp    2710.349
            infl          0.000
            unemp         5.800
1959-04-01  realgdp    2778.801
            infl          2.340
            unemp         5.100
1959-07-01  realgdp    2775.488
            infl          2.740
            unemp         5.300
1959-10-01  realgdp    2785.204
dtype: float64

- want to get rid of hierarchical index
- may want to provide a column name for data

In [19]:
# Give the three columns a name
wide_data.columns.name = "attribute"
wide_data[:10]

attribute,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2
1960-04-01,2834.390,0.14,5.2
1960-07-01,2839.022,2.70,5.6
1960-10-01,2802.616,1.21,6.3
1961-01-01,2819.264,-0.40,6.8


In [20]:
long_data = (wide_data.stack().reset_index().rename(columns={0: "value"}))
long_data1 = wide_data.stack()
long_data2 = long_data1.reset_index()
#long_data1[:10]
#long_data2[:10]
long_data[:10]

,date,attribute,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


In [21]:
long_data = long_data.set_index('date')
long_data

,attribute,value
date,,
1959-01-01,realgdp,2710.349
1959-01-01,infl,0.000
1959-01-01,unemp,5.800
1959-04-01,realgdp,2778.801
1959-04-01,infl,2.340
...,...,...
2009-04-01,infl,3.370
2009-04-01,unemp,9.200
2009-07-01,realgdp,12990.341


### Move back from long format to wide with pivot


In [22]:
wide_data2 = long_data.reset_index().pivot(index="date", columns="attribute", values="value")
wide_data2.head()

attribute,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


#### Suppose we have more than one value column in long format
 let's see what happens to the pivot
 two values could be something like begin and end quarter data


In [23]:
long_data2 = long_data.copy()
long_data2["value2"] = np.random.standard_normal(len(long_data))
long_data2.head()

,attribute,value,value2
date,,,
1959-01-01,realgdp,2710.349,-0.471269
1959-01-01,infl,0.000,1.060442
1959-01-01,unemp,5.800,-0.591302
1959-04-01,realgdp,2778.801,-0.274806
1959-04-01,infl,2.340,-0.202400


In [24]:
# Now when we pivot back to the Wide format we get hierarchical columns
wide_data3 = long_data2.reset_index().pivot(index="date", columns="attribute")
wide_data3[:10]

value                    value2                    
attribute   infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  1.060442 -0.471269 -0.591302
1959-04-01  2.34  2778.801   5.1 -0.202400 -0.274806  1.025174
1959-07-01  2.74  2775.488   5.3  0.037971  1.447734  0.920642
1959-10-01  0.27  2785.204   5.6 -1.645770 -0.538923  1.470924
1960-01-01  2.31  2847.699   5.2 -0.369425  0.103983 -0.006323
1960-04-01  0.14  2834.390   5.2  0.724132 -0.203633  0.783037
1960-07-01  2.70  2839.022   5.6 -0.647227  0.221716  0.301258
1960-10-01  1.21  2802.616   6.3  1.152052  0.462100 -2.387635
1961-01-01 -0.40  2819.264   6.8 -0.463005  0.954218 -0.902657
1961-04-01  1.47  2872.005   7.0 -0.812235 -1.305851 -1.284319

In [25]:
# We can select one portion of the hierarchy
wide_data3["value2"].head()

attribute,infl,realgdp,unemp
date,,,
1959-01-01,1.060442,-0.471269,-0.591302
1959-04-01,-0.202400,-0.274806,1.025174
1959-07-01,0.037971,1.447734,0.920642
1959-10-01,-1.645770,-0.538923,1.470924
1960-01-01,-0.369425,0.103983,-0.006323


In [26]:
# When we compute summary statistics we get them grouped by the hierarchy
# We can get the means for the two different value grouops
wide_data3.mean(axis=0)

# wide_data3.mean(axis=1) would sum up all values in a row & divide by number of columns

        attribute
value   infl            3.961330
        realgdp      7221.171901
        unemp           5.884729
value2  infl           -0.043487
        realgdp         0.108973
        unemp          -0.002122
dtype: float64

In [27]:
# Double check the calculations for value & infl
print(wide_data3["value"]["infl"].sum())
print(wide_data3.shape[0])
print(wide_data3["value"]["infl"].sum()/wide_data3.shape[0])

804.15
203
3.9613300492610835


#### A hiearchical index followed by an unstack can do the same thing as pivot

In [28]:
unstacked_long = long_data.reset_index().set_index(["date", "attribute"]).unstack(level="attribute")
unstacked1 = long_data.reset_index().set_index(["date", "attribute"])
unstacked2 = unstacked1.reset_index()
#unstacked1.head()
#unstacked2.head()
unstacked_long.head()

value                
attribute   infl   realgdp unemp
date                            
1959-01-01  0.00  2710.349   5.8
1959-04-01  2.34  2778.801   5.1
1959-07-01  2.74  2775.488   5.3
1959-10-01  0.27  2785.204   5.6
1960-01-01  2.31  2847.699   5.2

#### pd.pivot_table is a generalization of pivot

This was not covered in the reading (comes later), but I introduce now

pivot_table is a can handle duplicate values for one pivoted index/column pair.
     Specifically, you can give pivot_table a list of aggregation functions using keyword argument aggfunc.
      The default aggfunc of pivot_table is numpy.mean.
pivot_table also supports using multiple columns for the index and column of the pivoted table.
A hierarchical index will be automatically generated for you.

In [29]:
pd.pivot_table?

In [30]:
# In the simple case they are the same
pivoted1 = pd.pivot(long_data.reset_index(),index='date', columns='attribute', values='value')
pivoted2 = pd.pivot_table(long_data.reset_index(),index='date', columns='attribute', values='value')

# Now move back to the wide format
#
pivoted1
# pivoted2

attribute,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2
...,...,...,...
2008-07-01,-3.16,13324.600,6.0
2008-10-01,-8.79,13141.920,6.9
2009-01-01,0.94,12925.410,8.1


In [31]:
table = pd.pivot_table(data=original_data,index=['year'])
table

,cpi,infl,m1,pop,quarter,realcons,realdpi,realgdp,realgovt,realint,realinv,tbilrate,unemp
year,,,,,,,,,,,,,
1959.0,29.212500,1.337500,140.475,178.25475,2.5,1736.650000,1913.575000,2762.46050,481.66450,1.472500,296.584750,3.512500,5.450000
1960.0,29.670000,1.590000,140.450,181.12325,2.5,1784.350000,1964.000000,2830.93175,468.42725,1.120000,296.503250,2.707500,5.575000
1961.0,29.937500,0.667500,143.575,184.11225,2.5,1821.175000,2030.700000,2896.87950,488.13275,1.725000,294.585250,2.395000,6.700000
1962.0,30.312500,1.322500,146.975,186.93700,2.5,1911.225000,2129.625000,3072.39000,529.69400,1.467500,331.980500,2.790000,5.550000
1963.0,30.715000,1.630000,151.825,189.62950,2.5,1989.875000,2209.525000,3206.70825,530.07525,1.582500,354.271750,3.207500,5.650000
1964.0,31.092500,1.092500,157.875,192.24700,2.5,2108.400000,2368.700000,3392.31475,523.11800,2.477500,383.491250,3.567500,5.175000
1965.0,31.622500,1.902500,165.050,194.63700,2.5,2241.775000,2514.650000,3610.12750,522.62600,2.112500,437.300250,4.012500,4.525000
1966.0,32.620000,3.150000,171.300,196.87550,2.5,2368.975000,2647.275000,3845.34200,580.50150,1.725000,475.826500,4.875000,3.800000
1967.0,33.575000,3.582500,179.550,199.00925,2.5,2439.925000,2763.425000,3942.52300,638.46250,0.747500,454.066750,4.330000,3.825000


But we can aggregate with pivot table

In [32]:
# double check the results of pivot_table
original_data[original_data['year']==1959]['infl']

0    0.00
1    2.34
2    2.74
3    0.27
Name: infl, dtype: float64

Double check the calculations

In [33]:
original_data[original_data['year']==1959]['infl'].sum()/4

1.3375

Run another pivot table, this time on quarter for the fun of it

In [34]:
table = pd.pivot_table(data=original_data,index=['quarter'])
table

,cpi,infl,m1,pop,realcons,realdpi,realgdp,realgovt,realint,realinv,tbilrate,unemp,year
quarter,,,,,,,,,,,,,
1.0,104.182706,3.805882,661.198039,239.043294,4791.407843,5270.654902,7168.953059,658.234294,1.440000,1004.218725,5.301569,5.878431,1984.0
2.0,105.196020,4.128235,668.500000,239.696667,4826.603922,5317.370588,7227.666451,664.249824,1.126863,1013.555451,5.255490,5.894118,1984.0
3.0,106.145824,4.129216,676.405882,240.441078,4868.015686,5352.286275,7278.214667,669.190314,1.186863,1019.084176,5.315294,5.911765,1984.0
4.0,104.772660,3.778400,665.560000,239.715400,4814.942000,5301.678000,7209.627060,661.606360,1.597400,1014.631760,5.376000,5.854000,1983.5
